In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-sep-2021/test.csv')

In [ ]:
print(len(train))
print(len(test))

In [ ]:
# Huge data; therefore, using 20 percent of the data for initial model build
# train = train.sample(frac = 0.2, axis = 0)

In [ ]:
len(train)

In [ ]:
train['claim'].isna().sum()

In [ ]:
y = train['claim']
X = train.drop('claim', axis = 1)

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size = 0.8,
                                                               test_size = 0.2, random_state = 0)

In [ ]:
print(len(train.columns))

In [ ]:
# Types of columns
train.dtypes.describe()
# 118 - float
# 2 - integer

In [ ]:
# Lets check cardinality of columns to check if there are any categorical columns?
# Chances are less as all predictor columns are float except id
cardinality = [train[col].nunique() for col in train.columns.tolist()[:-1]]
cardinality.sort()
cardinality[:5]
# No categorical features

In [ ]:
train.isna().sum()

In [ ]:
imputer = SimpleImputer()
X_train_full = imputer.fit_transform(X_train_full)
X_valid_full = imputer.transform(X_valid_full)
X_test_full = imputer.transform(test)

In [ ]:
encoder = OrdinalEncoder()
y_train = pd.DataFrame(y_train)
y_valid = pd.DataFrame(y_valid)
y_train = encoder.fit_transform(y_train)
y_valid = encoder.transform(y_valid)

In [ ]:
y_train = y_train.ravel()
y_valid = y_valid.ravel()

In [ ]:
# Lets create a baseline xgboost model and then the model will be improved with feature engineering
model = XGBClassifier(n_estimators = 1000, use_label_encoder = False)
model.fit(X_train_full, y_train, early_stopping_rounds = 5,
         eval_set = [(X_valid_full, y_valid)], eval_metric = 'auc')

In [ ]:
predictions = model.predict_proba(X_test_full)[:, 1]

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'claim' : predictions})

In [ ]:
submission.to_csv('submission.csv', index = False)